# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Big Data** </center>
---
### <center> **Spring 2025** </center>
---
### <center> **Examples on storage solutions for Big Data (files)** </center>
---
**Profesor**: Dr. Pablo Camarillo Ramirez

In [2]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Storage-Solutions") \
    .master("spark://078b2e28e517:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

### Car rental service

In [4]:
from team_name.spark_utils import SparkUtils
agencies_schema = SparkUtils.generate_schema([("agency_id", "string"), ("agency_info", "string")])

agencies_df = spark.read \
                .schema(agencies_schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/rentals_dataset/agencies.csv")

agencies_df.printSchema()

agencies_df.show(5, truncate=False)

In [5]:
from pyspark.sql.functions import get_json_object
agencies_df = agencies_df.withColumn("agency_name", get_json_object(agencies_df.agency_info, "$.agency_name"))
agencies_df.show(truncate=False)

In [6]:
brands_schema = SparkUtils.generate_schema([("brand_id", "integer"), ("brand_info", "string")])
brands_df = spark.read.option("header", "true").schema(brands_schema).csv("/home/jovyan/notebooks/data/rentals_dataset/brands.csv")
brands_df.printSchema()
brands_df = brands_df.withColumn("brand_name", get_json_object(brands_df.brand_info, "$.brand_name"))
brands_df.show(5, truncate=False)

In [7]:
cars_schema = SparkUtils.generate_schema([("car_id", "integer"), ("car_info", "string")])
cars_df = spark.read.option("header", "true").schema(cars_schema).csv("/home/jovyan/notebooks/data/rentals_dataset/cars.csv")
cars_df.printSchema()
cars_df = cars_df.withColumn("car_name", get_json_object(cars_df.car_info, "$.car_name")) \
                .withColumn("brand_id", get_json_object(cars_df.car_info, "$.brand_id")) 
cars_df.show(5, truncate=False)

In [8]:
customers_schema = SparkUtils.generate_schema([("customer_id", "integer"), ("customer_info", "string")])
customers_df = spark.read.option("header", "true").schema(customers_schema).csv("/home/jovyan/notebooks/data/rentals_dataset/customers.csv")
customers_df.printSchema()
customers_df = customers_df.withColumn("customer_name", get_json_object(customers_df.customer_info, "$.customer_name"))
customers_df.show(5, truncate=True)

In [9]:
rental_cars_schema = SparkUtils.generate_schema([("rental_id", "integer"), ("rental_info", "string")])
rental_cars_df = spark.read.option("header", "true").schema(rental_cars_schema).csv("/home/jovyan/notebooks/data/rentals_dataset/rentals/")
rental_cars_df.printSchema()
rental_cars_df.show(5, truncate=False)

In [10]:
rental_cars_df = rental_cars_df.withColumn("car_id", get_json_object(rental_cars_df.rental_info, '$.car_id')) \
                            .withColumn("customer_id", get_json_object(rental_cars_df.rental_info, '$.customer_id')) \
                            .withColumn("agency_id", get_json_object(rental_cars_df.rental_info, '$.agency_id'))

rental_cars_df.show(5, truncate=False)

In [11]:
rental_cars_df = rental_cars_df.join(cars_df, rental_cars_df.car_id == cars_df.car_id, "inner") \
                                .join(agencies_df, rental_cars_df.agency_id == agencies_df.agency_id, "inner") \
                                .join(customers_df, rental_cars_df.customer_id == customers_df.customer_id, "inner")

rental_cars_df.show(5, truncate=False)

In [12]:
# Final result
rental_cars_df = rental_cars_df.select("rental_id", "car_name", "agency_name", "customer_name")

In [13]:
rental_cars_df.show(5, truncate=False)

In [14]:
rental_cars_df.createOrReplaceTempView("rentals")

In [15]:
spark.sql("SELECT rental_id, customer_name FROM rentals").show(5)

In [16]:
spark.sql("SELECT agency_name, count(*) as rentals_count FROM rentals GROUP BY agency_name").show()

#### Register temporal views

In [17]:
agencies_df.createOrReplaceTempView("agencies")
brands_df.createOrReplaceTempView("brands")
customers_df.createOrReplaceTempView("customers")
rental_cars_df.createOrReplaceTempView("rentals")
cars_df.createOrReplaceTempView("cars")

#### Identify the most popular car brands by the number of rentals

In [18]:
# Re create cars view
cars_df = spark.read.option("header", "true").schema(cars_schema).csv("/home/jovyan/notebooks/data/rentals_dataset/cars.csv")
cars_df.createOrReplaceTempView("cars")
cars_df = spark.sql("""
                    SELECT c.car_id, 
                           get_json_object(c.car_info, '$.car_name') AS car_name,
                           get_json_object(b.brand_info, '$.brand_name') AS brand_name
                    FROM cars c 
                    JOIN brands b
                    ON get_json_object(c.car_info, '$.brand_id') = b.brand_id
                    """)
cars_df.createOrReplaceTempView("cars")
spark.sql("SELECT * FROM cars").show(5)

In [19]:
# Re create rentals view
rental_cars_df = spark.read.option("header", "true").schema(rental_cars_schema).csv("/home/jovyan/notebooks/data/rentals_dataset/rentals/")
rental_cars_df.createOrReplaceTempView("rentals")
rental_cars_df = spark.sql("""
            SELECT r.rental_id,
                   c.car_name,
                   c.brand_name,
                   a.agency_name,
                   cus.customer_name
            FROM rentals r
            JOIN cars c ON get_json_object(r.rental_info, '$.car_id') = c.car_id
            JOIN agencies a ON get_json_object(r.rental_info, '$.agency_id') = a.agency_id
            JOIN customers cus ON get_json_object(r.rental_info, '$.customer_id') = cus.customer_id
          """)
rental_cars_df.createOrReplaceTempView("rentals")

spark.sql("""
          SELECT brand_name, count(*) AS rental_count FROM rentals 
          GROUP BY brand_name ORDER BY rental_count DESC LIMIT 3
          """).show()

#### Find the top 5 customers

In [20]:
spark.sql("""
            SELECT customer_name, count(*) AS rental_count FROM rentals GROUP BY customer_name ORDER BY rental_count DESC LIMIT 5
          """).show()

#### Find which car generate the most revenue

In [21]:
spark.sql("SELECT car_name, count(*) AS rental_count FROM rentals GROUP BY car_name ORDER BY rental_count DESC LIMIT 5").show()

#### Persist data

In [22]:
rental_cars_df.write \
                .mode("overwrite") \
                .partitionBy("agency_name") \
                .parquet("/home/jovyan/notebooks/data/rentals_output/")

In [23]:
# Stop the SparkContext
sc.stop()